# tqdm и joblib

In [3]:
from joblib import Parallel, delayed
from tqdm import tqdm
from tqdm_joblib import tqdm_joblib

Создадим функцию, которая будет моделировать сложные и долгие вычисления

In [4]:
import time

def myfun(x):
    time.sleep(5)
    return x**2

### Обычный tqdm

In [5]:
results = Parallel(n_jobs=4)(delayed(myfun)(i) for i in tqdm(range(10)))

100%|██████████| 10/10 [00:05<00:00,  1.81it/s]


In [6]:
results

[0, 1, 4, 9, 16, 25, 36, 49, 64, 81]

На первый взгляд работает хорошо, но можно заметить, что прогресс бар заполняется на 100% еще до того, как ячейка закончила выполнение.\
Это происходит из-за того, что прогресс бар смотрит, когда задача была взята в работу, а не когда выполнена.\
Для решения этой проблемы рассмотрим следующие конструкции:

### Обычный tqdm, но в основном процессе

Сделаем так, чтобы `Parallel` возвращал значения, как генератор, тогда можно сделать такую конструкцию:

In [14]:
results = [r for r in tqdm(Parallel(return_as="generator", n_jobs=4)(delayed(myfun)(i) for i in range(10)), total=10)]

100%|██████████| 10/10 [00:15<00:00,  1.50s/it]


In [8]:
results

[0, 1, 4, 9, 16, 25, 36, 49, 64, 81]

Проблема решена!\
Теперь наш `tqdm` работает, как должен и прибавляет проценты только, когда задачи выполнены.

### tqdm_joblib

Но у `joblib` есть еще и готовое решение:

In [17]:
data = list(range(10))

with tqdm_joblib(total=len(data)) as progress_bar:
    results = Parallel(n_jobs=4)(
        delayed(myfun)(i) for i in data
    )

  0%|          | 0/10 [00:00<?, ?it/s]

Но VS Code не поддерживает вывод в формате `tqdm_joblib`, но в Google Colab все работает хорошо.

In [13]:
results

[0, 1, 4, 9, 16, 25, 36, 49, 64, 81]